<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import gc
import numpy as np
import pandas as pd
from src.util import *
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from time import time
import datetime
from scipy.stats import ks_2samp

In [2]:
%%time
train_identity = pd.read_csv("data/train_identity.csv", )
train_transaction = pd.read_csv("data/train_transaction.csv", )
test_identity = pd.read_csv("data/test_identity.csv", )
test_transaction = pd.read_csv("data/test_transaction.csv",)

Wall time: 1min 1s


In [3]:
base_columns = list(train_transaction.columns) + list(train_identity.columns)
# base model feature improtance
feat = ['card2', 'C13', 'card1', 'TransactionAmt', 'C1', 'addr1', 'D15',
        'D2', 'P_emaildomain', 'C14', 'card5', 'C11', 'V45', 'D8',
        'card3', 'V313', 'D1', 'id_02', 'R_emaildomain', 'card6',
        'id_20', 'D4', 'D10', 'DeviceInfo', 'C2', 'id_01', 'V310',
        'V62', 'C12', 'dist1', 'V87', 'M4', 'V283', 'V281', 'V294',
        'V258', 'C8', 'V53', 'id_09', 'V314', 'V38', 'id_30', 'V315',
        'C6', 'V317', 'id_33', 'V312', 'V189', 'id_19', 'C4']

* Group same mobile phone company with different build into same group
* Fill missing values with unknown device

In [4]:
train_identity['DeviceInfo'] = train_identity['DeviceInfo'].fillna(
    'unknown_device').str.lower()
test_identity['DeviceInfo'] = test_identity['DeviceInfo'].fillna(
    'unknown_device').str.lower()

train_identity['DeviceInfo_c'] = train_identity['DeviceInfo']
test_identity['DeviceInfo_c'] = test_identity['DeviceInfo']

device_match_dict = {
    'sm': 'sm-',
    'sm': 'samsung',
    'huawei': 'huawei',
    'moto': 'moto',
    'rv': 'rv:',
    'trident': 'trident',
    'lg': 'lg-',
    'htc': 'htc',
    'blade': 'blade',
    'windows': 'windows',
    'lenovo': 'lenovo',
    'linux': 'linux',
    'f3': 'f3',
    'f5': 'f5'
}

for dev_type_s, dev_type_o in device_match_dict.items():
    train_identity['DeviceInfo_c'] = train_identity['DeviceInfo_c'].apply(
        lambda x: dev_type_s if dev_type_o in x else x)
    test_identity['DeviceInfo_c'] = test_identity['DeviceInfo_c'].apply(
        lambda x: dev_type_s if dev_type_o in x else x)

train_identity['DeviceInfo_c'] = train_identity['DeviceInfo_c'].apply(
    lambda x: 'other_d_type' if x not in device_match_dict else x)
test_identity['DeviceInfo_c'] = test_identity['DeviceInfo_c'].apply(
    lambda x: 'other_d_type' if x not in device_match_dict else x)

* Group same desktop os company with different build into same group

In [5]:
train_identity['id_30'] = train_identity['id_30'].fillna(
    'unknown_device').str.lower()
test_identity['id_30'] = test_identity['id_30'].fillna(
    'unknown_device').str.lower()

train_identity['id_30_c'] = train_identity['id_30']
test_identity['id_30_c'] = test_identity['id_30']

device_match_dict = {
    'ios':'ios',
    'windows':'windows',
    'mac':'mac',
    'android':'android'
}

for dev_type_s, dev_type_o in device_match_dict.items():
    train_identity['id_30_c'] = train_identity['id_30_c'].apply(
        lambda x: dev_type_s if dev_type_o in x else x)
    test_identity['id_30_c'] = test_identity['id_30_c'].apply(
        lambda x: dev_type_s if dev_type_o in x else x)

train_identity['id_30_v'] = train_identity['id_30'].apply(
    lambda x: ''.join([i for i in x if i.isdigit()]))
test_identity['id_30_v'] = test_identity['id_30'].apply(
    lambda x: ''.join([i for i in x if i.isdigit()]))

train_identity['id_30_v'] = np.where(
    train_identity['id_30_v'] != '', train_identity['id_30_v'], 0).astype(int)
test_identity['id_30_v'] = np.where(
    test_identity['id_30_v'] != '', test_identity['id_30_v'], 0).astype(int)

* make browser build and build number as seperate feature

In [6]:
train_identity['id_31'] = train_identity['id_31'].fillna(
    'unknown_br').str.lower()
test_identity['id_31'] = test_identity['id_31'].fillna(
    'unknown_br').str.lower()

train_identity['id_31'] = train_identity['id_31'].apply(
    lambda x: x.replace('webview', 'webvw'))
test_identity['id_31'] = test_identity['id_31'].apply(
    lambda x: x.replace('webview', 'webvw'))

train_identity['id_31'] = train_identity['id_31'].apply(
    lambda x: x.replace('for', ' '))
test_identity['id_31'] = test_identity['id_31'].apply(
    lambda x: x.replace('for', ' '))



In [7]:
browser_list = set(
    list(train_identity['id_31'].unique()) + list(test_identity['id_31'].unique()))
browser_list2 = []
for item in browser_list:
    browser_list2 += item.split(' ')
browser_list2 = list(set(browser_list2))

browser_list3 = []
for item in browser_list2:
    browser_list3 += item.split('/')
browser_list3 = list(set(browser_list3))

for item in browser_list3:
    train_identity['id_31_e_'+item] = np.where(
        train_identity['id_31'].str.contains(item), 1, 0).astype(np.int8)
    test_identity['id_31_e_'+item] = np.where(
        test_identity['id_31'].str.contains(item), 1, 0).astype(np.int8)
    if train_identity['id_31_e_'+item].sum() < 100:
        del train_identity['id_31_e_'+item], test_identity['id_31_e_'+item]

train_identity['id_31_v'] = train_identity['id_31'].apply(
    lambda x: ''.join([i for i in x if i.isdigit()]))
test_identity['id_31_v'] = test_identity['id_31'].apply(
    lambda x: ''.join([i for i in x if i.isdigit()]))

train_identity['id_31_v'] = np.where(
    train_identity['id_31_v'] != '', train_identity['id_31_v'], 0).astype(int)
test_identity['id_31_v'] = np.where(
    test_identity['id_31_v'] != '', test_identity['id_31_v'], 0).astype(int)

In [8]:
## Merge identity and transactions columns
temp_train_df = train_transaction[['TransactionID']]
temp_train_df = temp_train_df.merge(train_identity, on=['TransactionID'], how='left')
del temp_train_df['TransactionID']
train_merge = pd.concat([train_transaction, temp_train_df], axis=1)
del temp_train_df

temp_test_df = test_transaction[['TransactionID']]
temp_test_df = temp_test_df.merge(test_identity, on=['TransactionID'], how='left')
del temp_test_df['TransactionID']
test_merge = pd.concat([test_transaction, temp_test_df], axis=1)


In [9]:
del train_transaction, train_identity, test_transaction, test_identity
gc.collect()

251

In [10]:
train_merge = reduce_mem_usage(train_merge)
test_merge = reduce_mem_usage(test_merge)
print(f"Merged training set shape: {train_merge.shape}")
print(f"Merged testing set shape: {test_merge.shape}")

Mem. usage decreased to 714.68 Mb (67.6% reduction)
Mem. usage decreased to 620.45 Mb (67.1% reduction)
Merged training set shape: (590540, 489)
Merged testing set shape: (506691, 488)


In [11]:
def high_duplicates_col(df, base_columns):
    duplicates = []
    i = 0
    for c1 in base_columns:
        i += 1
        for c2 in base_columns[i: ]:
            if c1 != c2:
                if (np.sum((df[c1].values == \
                            df[c2].values).astype(int)) / len(df)) > 0.95:
                    duplicates.append(c2)
    
    return list(set(duplicates))

In [12]:
def get_cols_to_drop(df, base_columns):
    many_null_cols = [col for col in base_columns if df[col].isnull().sum() / df.shape[0] > 0.9]
    big_top_value_cols = [col for col in base_columns if
                          df[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
    cols_to_drop = list(set(many_null_cols + big_top_value_cols + high_duplicates_col(df, base_columns)))
   
    if 'isFraud' in cols_to_drop:
        cols_to_drop.remove('isFraud')

    return cols_to_drop


def make_day_feature(df, timecol='TransactionDT'):
    """
    Creates a day of the week feature, encoded as 0-6. 
    
    Parameters:
    -----------
    df : pd.DataFrame
        df to manipulate.
    timecol : str
        Name of the time column in df.
    """
    days = df[timecol] / (3600*24)        
    encoded_days = np.floor(days-1) % 7
    return encoded_days

def make_hour_feature(df, timecol='TransactionDT'):
    """
    Creates an hour of the day feature, encoded as 0-23. 
    
    Parameters:
    -----------
    df : pd.DataFrame
        df to manipulate.
    timecol : str
        Name of the time column in df.
    """
    hours = df[timecol] / (3600)        
    encoded_hours = np.floor(hours) % 24
    return encoded_hours

In [13]:
cols_to_drop = get_cols_to_drop(train_merge, base_columns)
print(cols_to_drop)

['V112', 'V315', 'V293', 'id_25', 'V135', 'V136', 'V284', 'V298', 'V300', 'V316', 'V111', 'dist2', 'V105', 'V113', 'V104', 'id_24', 'id_22', 'V117', 'V121', 'V125', 'V320', 'V103', 'V109', 'V118', 'V295', 'V303', 'V119', 'V134', 'V106', 'V281', 'V120', 'V290', 'V98', 'V102', 'V115', 'V137', 'V123', 'id_08', 'V309', 'id_18', 'V114', 'V321', 'V116', 'V133', 'V108', 'V301', 'V124', 'C3', 'V296', 'id_23', 'V122', 'V129', 'id_26', 'V304', 'V110', 'V107', 'id_21', 'V286', 'id_27', 'V297', 'V299', 'V311', 'V319', 'V305', 'V101', 'V289', 'id_07', 'V132', 'V318', 'D7']


In [ ]:
# ['V112', 'V315', 'V293', 'id_25', 'V135', 'V136', 'V284', 'V298', 'V300', 'V316', 
#  'V111', 'dist2', 'V105', 'V113', 'V104', 'id_24', 'id_22', 'V117', 'V121', 'V125', 
#  'V320', 'V103', 'V109', 'V118', 'V295', 'V303', 'V119', 'V134', 'V106', 'V281', 
#  'V120', 'V290', 'V98', 'V102', 'V115', 'V137', 'V123', 'id_08', 'V309', 'id_18', 
#  'V114', 'V321', 'V116', 'V133', 'V108', 'V301', 'V124', 'C3', 'V296', 'id_23', 
#  'V122', 'V129', 'id_26', 'V304', 'V110', 'V107', 'id_21', 'V286',  'id_27', 
#  'V297', 'V299', 'V311', 'V319', 'V305', 'V101', 'V289', 'id_07', 'V132', 'V318', 'D7']

* Around 40.8% of the transaction made by protonmail.com in `P_emaildomain` is fraudulent.
*  95% of the transaction made by protonmail.com in `R_emaildomain` is fraudulent.
* There are some links between fraud and the protonmail domain. Thus we can create features for this email domain 

In [15]:
fraud_count_r = train_merge.loc[train_merge['R_emaildomain'].isin(['protonmail.com']), 'isFraud'].value_counts()
num_entries_r = len(train_merge.loc[train_merge['R_emaildomain'].isin(['protonmail.com']), 'isFraud'])
fraud_percent_r = fraud_count_r / num_entries_r * 100
print(fraud_percent_r)

1    95.121951
0     4.878049
Name: isFraud, dtype: float64


In [16]:
fraud_count = train_merge.loc[train_merge['P_emaildomain'].isin(['protonmail.com']), 'isFraud'].value_counts()
num_entries = len(train_merge.loc[train_merge['P_emaildomain'].isin(['protonmail.com']), 'isFraud'])
fraud_percent = fraud_count / num_entries * 100
print(fraud_percent)

0    59.210526
1    40.789474
Name: isFraud, dtype: float64


In [17]:
train_merge['P_Isproton'] = (train_merge['P_emaildomain'] == 'protonmail.com')
train_merge['R_Isproton'] = (train_merge['R_emaildomain'] == 'protonmail.com')
test_merge['P_Isproton'] = (test_merge['P_emaildomain'] == 'protonmail.com')
test_merge['R_Isproton'] = (test_merge['R_emaildomain'] == 'protonmail.com')

In [18]:
train_merge['nulls_count'] = train_merge.isna().sum(axis=1)
test_merge['nulls_count'] = test_merge.isna().sum(axis=1)

In [19]:
# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum',
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft',
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft',
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other',
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink',
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other',
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo',
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other',
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other',
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other',
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}
us_emails = ['gmail', 'net', 'edu']

for col in ['P_emaildomain', 'R_emaildomain']:
    train_merge[col + '_bin'] = train_merge[col].map(emails)
    test_merge[col + '_bin'] = test_merge[col].map (emails)
    
    train_merge[col + '_suffix'] = train_merge[col].map(lambda x: str(x).split('.')[-1])
    test_merge[col + '_suffix'] = test_merge[col].map(lambda x: str(x).split('.')[-1])
    
    train_merge[col + '_suffix'] = train_merge[col + '_suffix'].map(
        lambda x: x if str(x) not in us_emails else 'us')
    test_merge[col + '_suffix'] = test_merge[col + '_suffix']. map(
        lambda x: x if str(x) not in us_emails else 'us')
 


* Time of day has some dependency on the fraudulent transaction 
* Decimal part of the transaction amount maybe useful features
* add client uID based on Card features and addr columns. need to remove it before modelling but can use it for aggregations features
* Too many unique values for TransactionAmt, thus not generalize well. Use aggregations of features to reduce the noise

In [20]:
# https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-feature
train_merge['Transaction_day'] = make_day_feature(train_merge)
test_merge['Transaction_day'] = make_day_feature(test_merge)
train_merge['Transaction_hour'] = make_hour_feature(train_merge)
test_merge['Transaction_hour'] = make_hour_feature(test_merge)

In [21]:
train_merge['uid'] = train_merge['card1'].astype(str) + \
    '_' + train_merge['card2'].astype(str) + \
    '_' + train_merge['card3'].astype(str) + \
    '_' + train_merge['card4'].astype(str)
test_merge['uid'] = test_merge['card1'].astype(str) + \
    '_' + test_merge['card2'].astype(str) + \
    '_' + test_merge['card3'].astype(str) + \
    '_' + test_merge['card4'].astype(str)

train_merge['uid2'] = train_merge['uid'].astype(str) + \
    '_' + train_merge['addr1'].astype(str) + '_' + \
    train_merge['addr2'].astype(str)
test_merge['uid2'] = test_merge['uid'].astype(str) +  \
    '_' + test_merge['addr1'].astype(str) + '_' + \
    test_merge['addr2'].astype(str)


In [22]:
card_uid_cols = ['card1', 'card2', 'card3', 'card5', 'uid', 'uid2']

for col in card_uid_cols:
    for agg_type in ['mean', 'std']:
        new_col_name = col + '_TransactionAmt_' + agg_type
        temp_df = pd.concat([train_merge[[col, 'TransactionAmt']],
                             test_merge[[col, 'TransactionAmt']]])
        temp_df = temp_df.groupby([col])['TransactionAmt'].agg(
            [agg_type]).reset_index().rename(columns={agg_type: new_col_name})
        temp_df.index = list(temp_df[col])
        temp_df = temp_df[new_col_name].to_dict()
        train_merge[new_col_name] = train_merge[col].map(temp_df)
        test_merge[new_col_name] = test_merge[col].map(temp_df)

In [23]:
m_cols = ['M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9']
train_merge['M_sum'] = train_merge[m_cols].sum(axis=1).astype(np.int8)
test_merge['M_sum'] = test_merge[m_cols].sum(axis=1).astype(np.int8)

train_merge['M_nulls'] = train_merge[m_cols].isna().sum(axis=1).astype(np.int8)
test_merge['M_nulls'] = test_merge[m_cols].isna().sum(axis=1).astype(np.int8)

* C columns are counts based on client information such as how many addresses are found to be associated with the payment card, etc
* All of the c columns are dominated by either 0.0 or 1.0, some of the values are abusrdly large.  Its not common for clients to have more than 30 address for the payment card. So we can create feature to identify whether the transaction belongs to common value counts of C features 
* We can create some aggregate features based on these information

In [24]:
c_cols = train_merge.iloc[:,17:31].columns

train_merge['C_sum'] = 0
test_merge['C_sum'] = 0

train_merge['C_null'] = 0
test_merge['C_null'] = 0

for cols in c_cols:
    train_merge['C_sum'] += np.where(train_merge[col] == 1, 1, 0)
    test_merge['C_sum'] += np.where(test_merge[col] == 1, 1, 0)
    
    train_merge['C_null'] += np.where(train_merge[col] == 0, 1, 0)
    test_merge['C_null'] += np.where(test_merge[col] == 0, 1, 0)
    
    valid_values = train_merge[col].value_counts()
    valid_values = valid_values[valid_values>1000]
    valid_values = list(valid_values.index)
    
    train_merge[col + '_valid'] = np.where(train_merge[col].isin(valid_values), 1, 0)
    test_merge[col + '_valid'] = np.where(test_merge[col].isin(valid_values), 1, 0)
    

In [25]:
freq_cols = ['card1', 'card2', 'card3', 'card5',
          'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14',
          'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9',
          'addr1', 'addr2',
          'dist1', 'dist2',
          'P_emaildomain', 'R_emaildomain',
          'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10',
          'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_24',
          'id_25', 'id_26', 'id_30', 'id_31', 'id_32', 'id_33',
          'DeviceInfo', 'DeviceInfo_c', 'id_30_c', 'id_30_v', 'id_31_v',
          ]

for col in freq_cols:
    temp_df = pd.concat([train_merge[[col]], test_merge[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()
    train_merge[col+'_fq_enc'] = train_merge[col].map(fq_encode)
    test_merge[col+'_fq_enc'] = test_merge[col].map(fq_encode)

In [26]:
# Label Encode object columns
for col in train_merge.columns:
    if train_merge[col].dtype == 'O':
        train_merge[col] = train_merge[col].astype(str)
        test_merge[col] = test_merge[col].astype(str)
        
        le = LabelEncoder()
        le.fit(list(train_merge[col]) + list(test_merge[col]))
        train_merge[col] = le.transform(train_merge[col])
        test_merge[col] = le.transform(test_merge[col])
        
        train_merge[col] = train_merge[col].astype('category')
        test_merge[col] = test_merge[col].astype('category')

In [27]:
features_check = []
columns_to_check = set(list(train_merge)).difference(base_columns)
for col in columns_to_check:
    features_check.append(ks_2samp(test_merge[col], train_merge[col])[1])

features_check = pd.Series(features_check, index=columns_to_check).sort_values() 
features_discard = list(features_check[features_check==0].index)


In [28]:
print(features_discard)

['id_31_e_4.0', 'id_31_e_58.0', 'id_31_e_15.0', 'id_31_e_62.0', 'M_nulls', 'id_31_e_16.0', 'id_31_fq_enc', 'D5_fq_enc', 'id_20_fq_enc', 'D7_fq_enc', 'id_31_e_generic', 'id_31_e_63.0', 'id_31_e_11.0', 'id_31_e_6.2', 'C12_fq_enc', 'id_31_e_desktop', 'nulls_count', 'D4_fq_enc', 'id_31_e_65.0', 'id_31_e_64.0', 'id_13_fq_enc', 'id_31_e_66.0', 'id_31_v_fq_enc', 'id_31_e_59.0', 'D3_fq_enc', 'D6_fq_enc', 'id_31_e_57.0', 'id_31_v', 'id_31_e_ie']


In [31]:
features_discard = features_discard + cols_to_drop

In [34]:
train_merge = train_merge.drop(features_discard, axis=1) 
test_merge = test_merge.drop(features_discard, axis=1)

In [36]:
train_merge.to_csv("preprocessed_data/train.csv", index=False)
test_merge.to_csv("preprocessed_data/test.csv", index=False)